# Dublin AFL Preprocessing

This notebook contains the code to prepare the AFL data for training, processing it from full videos hosted on S3 in Sydney, to frames that only contain the ball hosted in Dublin.

We assume that we have a `.xml` file with the annotations already uploaded to S3

### To clear the trash in Sagemaker

- Deleting the trash storage on Sagemaker
    - The path is `home/ec2-user/SageMaker/.Trash-1000/files`
    - `cd home/ec2-user/SageMaker/.Trash-1000/files`
    - Then `rm -rf *`

In [9]:
print("Let's begin:)")##

Let's begin:)


In [ ]:
# Install libraries
!pip install imageio[pyav]
!pip install imageio[ffmpeg]
!pip install opencv-python

In [3]:
# Import libraries
import boto3
import cv2
import imageio
import os
import shutil
import subprocess

from botocore.exceptions import NoCredentialsError
from typing import List, Dict
from sagemaker_utils import *

In [4]:
# Define constants
AUS_AFL_BUCKET: str = "australia-fov"
DUB_AFL_BUCKET: str = "dublin-afl-preprocessed"
ANNOTATIONS_FOLDER: str = "annotations"
S3_PREFIX: str = "train/unpacked_png/"  # Ensure it ends with "/"

if os.name == 'nt':
    TMP_DIR: str = "test-marvel"
else:
    TMP_DIR: str = "tmp"

VIDEOS_TO_PROCESS: List[str] = [
    "marvel/marvel-fov-3/20_08_2023/time_04_09_06_date_20_08_2023_.avi",
    # "marvel-fov-1\\20_08_2023\\marvel_1_time_04_09_04_date_20_08_2023_.avi",
    # "marvel-fov-8\\26_08_2023\\marvel_8_time_09_09_04_date_27_08_2023_.avi"
]

VIDEOS_TO_PROCESS = [path.replace("\\", "/") for path in VIDEOS_TO_PROCESS]

First, let's inspect our buckets

In [ ]:
# Ensure that each video path exists on s3
s3 = boto3.client('s3')

print("Checking video paths exist on S3...")
for video_path in VIDEOS_TO_PROCESS:
    try:
        s3.head_object(Bucket=AUS_AFL_BUCKET, Key=video_path)
        print(f"Video {video_path} exists on S3")
    except:
        print(f"Video {video_path} does not exist on S3!")

## Utility functions

Next, we define a bunch of helper function

Next, we'll define functions that we'll use to process the videos. We will only operate on one video at a time so we don't need too much storage space locally.

In [16]:
def clip_video_opencv(video_path: str, output_dir: str, clip_length: int = 60) -> None:
    """
    Clip a video into multiple segments of the given length using OpenCV.

    Parameters:
    - video_path: Path to the video file.
    - output_dir: Directory to store the clipped videos.
    - clip_length: Duration of each clip in seconds (default is 60 seconds).
    """

    # Check if video exists
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"Video at {video_path} does not exist!")

    # Open the video using OpenCV
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Error opening video at {video_path}")

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the number of frames needed for the specified clip length
    frames_per_clip = clip_length * fps

    # Create the output directory if it doesn't exist
    ensure_directory_exists(output_dir)

    # Get the video file name without the extension
    file_name = os.path.splitext(os.path.basename(video_path))[0]

    # Get the marvel caemra number
    camera_number = [part[-1] for part in video_path.split('/') if part.startswith("marvel-fov-")][0]

    clip_num = 0
    while True:
        output_file = os.path.join(output_dir, f"marvel_{camera_number}_{file_name}{clip_num}.mp4")

        # Define video writer for output
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_file, fourcc, fps, (int(cap.get(3)), int(cap.get(4))))

        # Extract frames and write to new video
        for _ in range(frames_per_clip):
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)

        # Release the current writer
        out.release()

        # If there were fewer frames left than half the desired clip length, we stop
        if ret and (total_frames - clip_num * frames_per_clip) < (0.5 * clip_length * fps):
            os.remove(output_file)
            break

        clip_num += 1

        # If we've reached the end of the video
        if not ret:
            break

    # Release video capture
    cap.release()


In [7]:
def extract_frames_from_video_cv2(file_path: str, fps: int = 30) -> None:
    """Given a video file path, extract frames using OpenCV (imageio is too slow for this)."""

    # Create a directory called "frames"
    file_name_without_extension = os.path.splitext(os.path.basename(file_path))[0]
    file_dir = os.path.dirname(file_path)

    frames_dir = create_directory(file_dir, directory_name=f'{file_name_without_extension}')

    if count_files_with_extension(frames_dir, '.png') > 1780:
        print(f'Frames already exist in {frames_dir}')
        return

    # Ensure the video path exists
    if not os.path.exists(file_path):
        raise FileNotFoundError(f'Video file not found at {file_path}')

    # Use OpenCV to read the video and save frames
    cap = cv2.VideoCapture(file_path)
    video_fps = int(cap.get(cv2.CAP_PROP_FPS))

    # We'll only save every nth frame to match the desired FPS (fps parameter)
    n = int(video_fps / fps)

    frame_num = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_num % n == 0:
            frame_file_path = os.path.join(frames_dir, f"frame_{frame_num // n:07d}.png")
            cv2.imwrite(frame_file_path, frame)
        frame_num += 1

    cap.release()
    print(f"Frames extracted to {frames_dir}")

In [6]:
import os
import boto3

AUS_AFL_BUCKET = "australia-fov"

def download_video_from_s3(video_path: str):
    """
    Download specified videos from the S3 bucket to local storage.
    If they already exist, ensure they have a reasonable size.

    Parameters:
    - video_path: Path of video to process.
    """
    video_path = video_path.replace("\\", "/")

    # Ensure the directory structure exists for the local file
    if not os.path.exists(os.path.dirname(video_path)):
        os.makedirs(os.path.dirname(video_path))

    # Ensure the video file exists on S3
    try:
        s3.head_object(Bucket=AUS_AFL_BUCKET, Key=video_path)
        print(f"Video {video_path} exists on S3")
    except:
        print(f"Video {video_path} does not exist on S3!")

    # Check if the file already exists in local storage
    if os.path.exists(video_path):
        # Check if file size is greater than 10 KB
        if os.path.getsize(video_path) > 10 * 1024:
            print(f"{video_path} already exists in local storage and has a valid size!")
            return
        else:
            print(f"{video_path} exists but is too small. Re-downloading...")
            os.remove(video_path)

    print(f"Downloading {video_path}...")
    try:
        # Download the file
        with open(video_path, 'wb') as f:
            s3.download_fileobj(AUS_AFL_BUCKET, video_path, f)
        print(f"{video_path} downloaded successfully!")
        print(f"File size in bytes: {get_file_size_in_bytes(video_path)}")
    except Exception as e:
        print(f"Error downloading {video_path}: {e}")
        if os.path.exists(video_path):
            os.remove(video_path)
        raise e

In [7]:
def remove_frames_without_ball(video_name: str, annotations_folder: str) -> None:
    xml_file_path = os.path.join(annotations_folder, f"{video_name}.xml")

    frame_nums: List[int] = get_frames_containing_ball(xml_file_path)
    file_names: List[str] = [get_frame_name_from_frame_number(frame_num) for frame_num in frame_nums]

    extracted_image_file: List[str] = find_files_with_ending(f"{TMP_DIR}/{video_name}", ".png")

    for file in extracted_image_file:
        file_name = os.path.basename(file)
        if file_name not in file_names:
            os.remove(file)

In [10]:
def upload_folder_to_s3(folder_path: str, bucket: str, prefix: str) -> None:
    """
    Uploads all .png files from a given folder to a specified S3 bucket and prefix.

    Parameters:
    - folder_path (str): Local path to the folder containing .png files
    - bucket (str): Name of the S3 bucket to upload to
    - prefix (str): S3 prefix (folder) to upload the files to

    Returns:
    - None
    """
    if not os.path.exists(folder_path):
        print(f"Folder {folder_path} does not exist!")
        return
    else:
        print(f"Uploading {folder_path} to {bucket}{prefix}")

    # Ensure prefix ends with a "/"
    if not prefix.endswith('/'):
        print("Adding '/' to prefix")
        prefix += '/'

    folder_name = os.path.basename(folder_path)

    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.endswith('.png'):
                local_path = os.path.join(root, filename)
                s3_path = prefix + folder_name + "/" + filename

                try:
                    s3.upload_file(local_path, bucket, s3_path)
                    print(f"Uploaded {filename} to {bucket}/{s3_path}")
                except NoCredentialsError:
                    print("Credentials not available")
                except Exception as e:
                    print(f"Error uploading {filename}. Error: {e}")

## Processing time

In [11]:
for video_path in VIDEOS_TO_PROCESS:
    # Download the videos to local storage, but check if they exist already first
    download_video_from_s3(video_path)

    # Clip video into 1 minute clips
    clip_video_opencv(video_path, output_dir=TMP_DIR, clip_length=60)

    # Extract frames from clipped videos
    for video in find_files_with_ending(TMP_DIR, '.mp4'):
        # Extract frames
        extract_frames_from_video_cv2(video)

        # Remove excess frames
        video_name = os.path.splitext(os.path.basename(video))[0]
        remove_frames_without_ball(video_name=video_name, annotations_folder=ANNOTATIONS_FOLDER)

        # Upload folders to S3
        folder_name = os.path.splitext(video)[0]
        upload_folder_to_s3(folder_path=folder_name, bucket=DUB_AFL_BUCKET, prefix=S3_PREFIX)

    shutil.rmtree(TMP_DIR)
    if not os.name == 'nt':
        shutil.rmtree(".Trash-1000/files")

Found 1 files with ending .mp4 in test-marvel
Uploading test-marvel\marvel_3_time_04_09_06_date_20_08_2023_0 to dublin-afl-preprocessedtrain/unpacked_png/
